# 네이버 데이터랩 인기 검색어 개별 수집 프로그램

## 프로그램 설명
네이버 데이터랩에서 인기 검색어를 자동으로 수집하고 엑셀 파일로 저장하는 프로그램입니다.

## 주요 기능
- 네이버 데이터랩 인기 검색어 자동 수집
- 500개 검색어 자동 수집 (25페이지)
- 카테고리 정보 자동 추출 (분야1, 분야2, 분야3, 분야4)
- 엑셀 파일 자동 저장 및 열기
- 기존 엑셀 파일에 데이터 추가 기능

## 수집 데이터
- 순위
- 인기 검색어
- 기간
- 분야1, 분야2, 분야3, 분야4 (카테고리 정보)

## 사용 방법
1. 프로그램 실행
2. 저장할 파일명 입력 (확장자 자동 추가)
3. 자동으로 데이터 수집 및 엑셀 저장
4. 프로그램 종료 후 엑셀 파일 자동 열기

## 필요 라이브러리
- selenium: 웹 크롤링
- pandas: 데이터 처리
- openpyxl: 엑셀 파일 처리

## 주의사항
- 네이버 데이터랩 페이지 구조 변경 시 수정이 필요할 수 있습니다
- 기존 엑셀 파일이 있으면 데이터를 추가합니다
- 500개 데이터 수집에는 시간이 소요됩니다

In [5]:
import pandas as pd
import os
import subprocess
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def setup_webdriver():
    """웹드라이버 실행 및 네이버 데이터랩 접속"""
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)
    driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")
    return driver

def get_category_info(driver):
    """현재 선택된 카테고리 정보 가져오기"""
    categories = {}
    category_base_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[{}]/span'
    for i in range(1, 5):
        try:
            category_text = driver.find_element(By.XPATH, category_base_xpath.format(i)).text.strip()
            if category_text:
                categories[f'분야{i}'] = category_text
        except:
            break
    return categories

def scrape_data(driver, categories):
    """데이터 수집 (총 500개 데이터, 25페이지 기준)"""
    data = []
    try:
        period = driver.find_element(By.CSS_SELECTOR, 'span.title_sub').text
    except:
        period = "알 수 없음"

    for page in range(25):
        time.sleep(1)
        for item in range(1, 21):
            try:
                path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{item}]/a'
                result = driver.find_element(By.XPATH, path).text
                rank, keyword = result.split('\n')
                row_data = [int(rank), keyword, period] + [categories.get(f'분야{i}', "") for i in range(1, 5)]
                data.append(row_data)
            except:
                continue
        try:
            if page < 24:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn_page_next'))
                )
                next_button.click()
        except:
            print(f"⚠ 페이지 {page + 1}에서 더 이상 이동할 수 없습니다.")
            break
    return data

def save_to_excel(file_name, data):
    """엑셀 파일에 데이터 저장"""
    df = pd.DataFrame(data, columns=['순위', '인기검색어', '기간', '분야1', '분야2', '분야3', '분야4'])
    sheet_name = "데이터"

    if os.path.exists(file_name):
        book = load_workbook(file_name)
        if sheet_name in book.sheetnames:
            with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                start_row = book[sheet_name].max_row
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=start_row)
        else:
            with pd.ExcelWriter(file_name, engine='openpyxl', mode='a') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"✅ 데이터 저장 완료: {file_name}")

def open_excel(file_name):
    """엑셀 파일 자동 열기 (프로그램 종료 후 실행)"""
    try:
        if os.name == "nt":  # Windows
            os.startfile(file_name)
        elif os.name == "posix":  # macOS, Linux
            subprocess.call(["open", file_name])  # macOS
            subprocess.call(["xdg-open", file_name])  # Linux
    except Exception as e:
        print(f"⚠ 파일을 자동으로 열 수 없습니다: {e}")

def main():
    # 파일명 입력받기 (확장자는 자동 추가)
    file_name = input("저장할 파일명을 입력하세요 (예: 네이버_데이터랩_수집): ").strip()
    if not file_name.lower().endswith('.xlsx'):
        file_name += ".xlsx"

    driver = setup_webdriver()

    while True:
        input("\n카테고리를 선택하고 조회 버튼을 클릭한 후, 엔터를 누르세요...")
        categories = get_category_info(driver)
        print(f"\n✅ 선택된 카테고리: {categories}")

        data = scrape_data(driver, categories)
        if data:
            save_to_excel(file_name, data)
        else:
            print("⚠ 데이터가 수집되지 않았습니다.")

        repeat = input("\n또 다른 카테고리를 스크랩하시겠습니까? (y/n): ").strip().lower()
        repeat = repeat.replace("ㅛ", "y").replace("ㅜ", "n")  # 한글 입력 대응
        
        if repeat != 'y':
            break

    driver.quit()
    open_excel(file_name)  # 프로그램 종료 후 엑셀 파일 열기
    print("\n🔄 프로그램 종료")

if __name__ == "__main__":
    main()


저장할 파일명을 입력하세요 (예: 네이버_데이터랩_수집): 베게

카테고리를 선택하고 조회 버튼을 클릭한 후, 엔터를 누르세요...

✅ 선택된 카테고리: {'분야1': '가구/인테리어', '분야2': '베개', '분야3': '3분류'}
✅ 데이터 저장 완료: 베게.xlsx

또 다른 카테고리를 스크랩하시겠습니까? (y/n): ㅛ

카테고리를 선택하고 조회 버튼을 클릭한 후, 엔터를 누르세요...

✅ 선택된 카테고리: {'분야1': '가구/인테리어', '분야2': '베개', '분야3': '3분류'}
✅ 데이터 저장 완료: 베게.xlsx

또 다른 카테고리를 스크랩하시겠습니까? (y/n): ㅛ

카테고리를 선택하고 조회 버튼을 클릭한 후, 엔터를 누르세요...

✅ 선택된 카테고리: {'분야1': '가구/인테리어', '분야2': '베개', '분야3': '3분류'}
⚠ 데이터가 수집되지 않았습니다.

또 다른 카테고리를 스크랩하시겠습니까? (y/n): ㅜ

🔄 프로그램 종료


In [1]:
"""
네이버 데이터랩 쇼핑인사이트 카테고리별 인기검색어 수집 프로그램
작성일: 2024-03-21
업데이트: 2025-04-17

주요 기능:
- 네이버 데이터랩의 쇼핑인사이트에서 카테고리별 인기검색어 데이터를 수집
- 수집된 데이터를 엑셀 파일로 저장
- 여러 카테고리 연속 수집 가능
- GUI 인터페이스 제공
- Jupyter Notebook 환경 지원
"""

import pandas as pd
import os
import subprocess
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import ipywidgets as widgets
from IPython.display import display, HTML
import threading

class ShoppingInsightGUI:
    def __init__(self):
        self.driver = None
        self.file_name = None
        self.create_widgets()
        
    def create_widgets(self):
        # 파일명 입력 위젯
        self.file_input = widgets.Text(
            description='파일명:',
            placeholder='네이버_데이터랩_수집'
        )
        
        # 버튼들
        self.start_button = widgets.Button(description='브라우저 시작')
        self.collect_button = widgets.Button(description='데이터 수집', disabled=True)
        self.stop_button = widgets.Button(description='프로그램 종료', disabled=True)
        
        # 상태 표시
        self.status = widgets.HTML(value='준비됨')
        
        # 이벤트 핸들러 연결
        self.start_button.on_click(self.on_start)
        self.collect_button.on_click(self.on_collect)
        self.stop_button.on_click(self.on_stop)
        
        # 위젯 표시
        display(HTML('<h2>네이버 쇼핑인사이트 데이터 수집기</h2>'))
        display(self.file_input)
        display(widgets.HBox([self.start_button, self.collect_button, self.stop_button]))
        display(self.status)

    def setup_webdriver(self):
        """웹드라이버 설정 및 초기화"""
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument("--disable-blink-features=AutomationControlled")
        self.driver = webdriver.Chrome(options=options)
        self.driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")

    def get_category_info(self):
        """현재 선택된 카테고리 정보 수집"""
        categories = {}
        category_base_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[{}]/span'
        for i in range(1, 5):
            try:
                category_text = self.driver.find_element(By.XPATH, category_base_xpath.format(i)).text.strip()
                if category_text:
                    categories[f'분야{i}'] = category_text
            except:
                break
        return categories

    def scrape_data(self, categories):
        """데이터 수집"""
        data = []
        try:
            period = self.driver.find_element(By.CSS_SELECTOR, 'span.title_sub').text
        except:
            period = "알 수 없음"

        for page in range(25):
            time.sleep(1)
            for item in range(1, 21):
                try:
                    path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{item}]/a'
                    result = self.driver.find_element(By.XPATH, path).text
                    rank, keyword = result.split('\n')
                    row_data = [int(rank), keyword, period] + [categories.get(f'분야{i}', "") for i in range(1, 5)]
                    data.append(row_data)
                except:
                    continue
            try:
                if page < 24:
                    next_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn_page_next'))
                    )
                    next_button.click()
            except:
                self.status.value = f"⚠ 페이지 {page + 1}에서 더 이상 이동할 수 없습니다."
                break
        return data

    def save_to_excel(self, data):
        """엑셀 파일 저장"""
        df = pd.DataFrame(data, columns=['순위', '인기검색어', '기간', '분야1', '분야2', '분야3', '분야4'])
        sheet_name = "데이터"

        if os.path.exists(self.file_name):
            book = load_workbook(self.file_name)
            if sheet_name in book.sheetnames:
                with pd.ExcelWriter(self.file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                    start_row = book[sheet_name].max_row
                    df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=start_row)
            else:
                with pd.ExcelWriter(self.file_name, engine='openpyxl', mode='a') as writer:
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            with pd.ExcelWriter(self.file_name, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        self.status.value = f"✅ 데이터 저장 완료: {self.file_name}"

    def on_start(self, b):
        """브라우저 시작 버튼 핸들러"""
        self.file_name = self.file_input.value.strip()
        if not self.file_name:
            self.status.value = "⚠ 파일명을 입력해주세요."
            return
            
        if not self.file_name.lower().endswith('.xlsx'):
            self.file_name += ".xlsx"
            
        self.setup_webdriver()
        self.collect_button.disabled = False
        self.stop_button.disabled = False
        self.start_button.disabled = True
        self.status.value = "✅ 브라우저가 시작되었습니다. 카테고리를 선택하고 조회 버튼을 클릭한 후 '데이터 수집' 버튼을 눌러주세요."

    def on_collect(self, b):
        """데이터 수집 버튼 핸들러"""
        self.status.value = "🔄 데이터 수집 중..."
        categories = self.get_category_info()
        self.status.value = f"✅ 선택된 카테고리: {categories}"
        
        data = self.scrape_data(categories)
        if data:
            self.save_to_excel(data)
        else:
            self.status.value = "⚠ 데이터가 수집되지 않았습니다."
        
        self.status.value += "<br>다른 카테고리를 선택하고 다시 '데이터 수집' 버튼을 누르거나, '프로그램 종료' 버튼을 눌러주세요."

    def on_stop(self, b):
        """프로그램 종료 버튼 핸들러"""
        if self.driver:
            self.driver.quit()
        self.collect_button.disabled = True
        self.stop_button.disabled = True
        self.start_button.disabled = False
        self.status.value = "🔄 프로그램이 종료되었습니다."

# GUI 실행
gui = ShoppingInsightGUI()

Text(value='', description='파일명:', placeholder='네이버_데이터랩_수집')

HTML(value='준비됨')